Human Footprint Impact on Biodiversity	bio.015	https://www.nature.com/articles/ncomms12558

http://datadryad.org/bitstream/handle/10255/dryad.131447/HumanFootprintv2.7z?sequence=2

In [1]:
import numpy as np
import boto3
import rasterio

import glob

import os
import sys
import threading

In [3]:
file_src_folder = "/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/"
file_names_folder = "Dryadv3/Maps/"
file_loc = file_src_folder + file_names_folder

In [4]:
files = glob.glob(file_loc + "/*.tif")

In [6]:
files

[]

Investigate whether all have same extent, resolution

In [5]:
metadata_store = []
for file in files:
    with rasterio.open(file) as src:
        p = src.profile
        description = [file, p["dtype"], p["height"], p["width"], p["nodata"]]
        windows = src.block_windows(1)
        for ix, win in windows:
            first = (ix, win)
            break
        description.append(first)
        metadata_store.append(description)
        
        

In [6]:
metadata_store

[['/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/Dryadv3/Maps/Built1994.tif',
  'int32',
  16382,
  36081,
  2147483647.0,
  ((0, 0), ((0, 128), (0, 128)))],
 ['/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/Dryadv3/Maps/Built2009.tif',
  'int32',
  16382,
  36081,
  2147483647.0,
  ((0, 0), ((0, 128), (0, 128)))],
 ['/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/Dryadv3/Maps/croplands1992.tif',
  'uint8',
  16382,
  36081,
  127.0,
  ((0, 0), ((0, 128), (0, 128)))],
 ['/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/Dryadv3/Maps/croplands2005.tif',
  'uint8',
  16382,
  36081,
  127.0,
  ((0, 0), ((0, 128), (0, 128)))],
 ['/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/Dryadv3/Maps/HFP1993.tif',
  'float32',
  16382,
  36081,
  -9999.0,
  ((0, 0), ((0, 128), (0, 128)))],
 ['/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/Dryadv3/Maps/HFP1993_int.tif',
  'uint8',
  16382,
  36081

In [46]:
for file in files:
    os.environ["ZFILELOC"] = file
    !gdalinfo $ZFILELOC

Driver: GTiff/GeoTIFF
Files: /Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/Dryadv3/Maps/Built1994.tif
       /Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/Dryadv3/Maps/Built1994.tif.ovr
       /Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/Dryadv3/Maps/Built1994.tif.aux.xml
Size is 36081, 16382
Coordinate System is:
PROJCS["World_Mollweide",
    GEOGCS["GCS_WGS_1984",
        DATUM["D_WGS_1984",
            SPHEROID["WGS_1984",6378137.0,298.257223563]],
        PRIMEM["Greenwich",0.0],
        UNIT["Degree",0.0174532925199433]],
    PROJECTION["Mollweide"],
    PARAMETER["False_Easting",0.0],
    PARAMETER["False_Northing",0.0],
    PARAMETER["Central_Meridian",0.0],
    UNIT["Meter",1.0]]
Origin = (-18040094.097520448267460,9018957.052787266671658)
Pixel Size = (1000.000000000000000,-1000.000000000000000)
Metadata:
  AREA_OR_POINT=Area
  DataType=Generic
Image Structure Metadata:
  COMPRESSION=LZW
  INTERLEAVE=BAND
Corner Coordin

In [5]:
os.environ["ZFILELOC"] = files[7]
!gdalinfo $ZFILELOC

IndexError: list index out of range

In [9]:
HFP2009 = "/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/Dryadv3/Maps/HFP2009.tif"
os.environ["ZFILELOC"] = HFP2009

!gdalinfo $ZFILELOC

os.environ["Z_FILE_TEMP"] = "./temp.vrt"

# Reproject
# http://www.gdal.org/gdalwarp.html

!gdalwarp -t_srs EPSG:4326 -of vrt $ZFILELOC $Z_FILE_TEMP

# Compress
# http://www.gdal.org/gdal_translate.html

HFP2009_reproj_compress = "./bio_015_human_footprint.tif"
os.environ["Z_FILE_DEST"] = HFP2009_reproj_compress

!gdal_translate -of GTiff -co COMPRESS=LZW $Z_FILE_TEMP $Z_FILE_DEST

!gdalinfo $Z_FILE_DEST

Driver: GTiff/GeoTIFF
Files: /Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/Dryadv3/Maps/HFP2009.tif
       /Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/Dryadv3/Maps/HFP2009.tif.ovr
       /Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/Dryadv3/Maps/HFP2009.tif.aux.xml
Size is 36081, 16382
Coordinate System is:
PROJCS["World_Mollweide",
    GEOGCS["GCS_WGS_1984",
        DATUM["D_WGS_1984",
            SPHEROID["WGS_1984",6378137.0,298.257223563]],
        PRIMEM["Greenwich",0.0],
        UNIT["Degree",0.0174532925199433]],
    PROJECTION["Mollweide"],
    PARAMETER["False_Easting",0.0],
    PARAMETER["False_Northing",0.0],
    PARAMETER["Central_Meridian",0.0],
    UNIT["Meter",1.0]]
Origin = (-18040094.097520448267460,9018957.052787266671658)
Pixel Size = (1000.000000000000000,-1000.000000000000000)
Metadata:
  AREA_OR_POINT=Area
  DataType=Generic
Image Structure Metadata:
  COMPRESSION=LZW
  INTERLEAVE=BAND
Corner Coordinates:


In [10]:
with rasterio.open("./bio_015_human_footprint.tif") as src:
    p = src.profile
    print(p)
    data = src.read(1)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -9999.0, 'width': 36390, 'height': 15683, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': (-179.9999999544725, 0.009892734568157884, 0.0, 89.89762321520959, 0.0, -0.009892734568157884), 'affine': Affine(0.009892734568157884, 0.0, -179.9999999544725,
       0.0, -0.009892734568157884, 89.89762321520959), 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}


In [12]:
data

array([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       ..., 
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.]], dtype=float32)

In [ ]:
# ## This causes trouble because of different data types
# # There are also different window sizes for two of the files

# outFile = "./trial_multilayering.tif"

# with rasterio.open(outFile, 'w', **kwargs) as dst:
#     for ix, file in enumerate(files):
#         print(file)
#         with rasterio.open(file) as src:
#             windows = src.block_windows(1)
#             nodata = src.meta["nodata"]
#             for idx, window in windows:
#                 window_data = src.read(1, window=window)

#                 # Source nodata value is a very small negative number
#                 # Converting in to zero for the output raster
#                 np.putmask(window_data, window_data == nodata, -9999)

#                 dst.write_band(ix+1,window_data, window=window)

In [4]:
s3 = boto3.client("s3")
#HFP2009_reproj_compress = "./bio_015_human_footprint.tif"
HFP2009 = "/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/Dryadv3/Maps/HFP2009.tif"
class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write(
                    "\r%s  %s / %s  (%.2f%%)" % (
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

bucket_name = "wri-public-data"
folder_dest = "resourcewatch/bio_015_human_footprint_impact_on_biodiversity/"
file_dest_name = "bio_015_human_footprint_impact_on_biodiversity.tif"

file_dest = folder_dest + file_dest_name

s3.upload_file(HFP2009, bucket_name, file_dest,
                         Callback=ProgressPercentage(HFP2009))

/Users/nathansuberi/Desktop/WRI_Programming/RW_Data/Biodiversity/Dryadv3/Maps/HFP2009.tif  233084809 / 233084809.0  (100.00%)